In [3]:
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
from lifelines import CoxPHFitter

In [2]:
#load main table
copd_tb_f = '../results/copd_table_post_filtering.tsv'
tb = pd.read_csv(copd_tb_f, sep='\t', index_col='eid')

#load diabetes
dm_f = 'diabetes_dates.tsv'
dm = pd.read_csv(dm_f, sep='\t', index_col='eid', usecols=['eid', 'diabetes'])

#load insulin self-report
insulin_f = 'ukb_insulin_self_report.tsv'
insulin = pd.read_csv(insulin_f, sep='\t', index_col='eid')

#join tables
tb = tb.join([dm, insulin])

#change dtypes of date columns
date_cols = [col for col in tb.columns if col.endswith('date')]
dtypes = {col: np.datetime64 for col in date_cols}
tb = tb.astype(dtypes)

tb

,sex,height,age_at_ac1,ethnicity,Townsend_DI,bmi,complex_chd,noncomplex_chd,isolated_aov,chd,...,event,hypertension_comb,smoking,smoking_status_0,smoking_status_1,smoking_status_2,TDI_binned,yrs_duration,diabetes,takes_insulin
eid,,,,,,,,,,,,,,,,,,,,,
3869328,1.0,176.0,64.0,1.0,0.611146,23.7926,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,"(-2.806, 0.646]",20.000000,0,0.0
1713188,0.0,176.0,63.0,1.0,-0.474842,35.8019,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,"(-2.806, 0.646]",20.000000,0,0.0
2429802,1.0,163.0,66.0,1.0,3.899080,31.3523,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,"(0.646, 4.097]",13.763176,1,0.0
2223778,0.0,166.0,55.0,1.0,0.099974,22.1730,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,"(-2.806, 0.646]",20.000000,0,0.0
4797749,1.0,165.0,57.0,1.0,-5.860310,32.3600,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,0.0,"(-6.258, -2.806]",20.000000,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2748576,0.0,160.0,68.0,1.0,-2.223210,24.0234,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,"(-2.806, 0.646]",20.000000,0,0.0
3689066,0.0,164.0,54.0,1.0,3.044200,22.6056,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,"(0.646, 4.097]",20.000000,0,0.0
1871700,1.0,166.0,53.0,0.0,3.753720,23.8424,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,"(0.646, 4.097]",20.000000,0,0.0


In [8]:
#define a new binary outcome based on diagnosis of diabetes or self-reported insulin use
dm_diag = tb['diabetes'] == 1
uses_insulin = tb['takes_insulin'] == 1
tb['diabetes_comb'] = np.where(dm_diag | uses_insulin, 1, 0)
tb['diabetes_comb'].value_counts()

0    471488
1     11662
Name: diabetes_comb, dtype: int64

In [9]:
#stratified cox regression
full_form_cat = (
    'sex + age_at_ac1 + bmi + ethnicity + diabetes_comb'
)

cat_strata = [
    'smoking',
    'hypertension_comb',
    'asthma',
    'TDI_binned'
]

formtypes = {
    'Fully Adjusted': (tb, full_form_cat, cat_strata)
}

results = {}
groups = ['All CHD', 'Isolated AoV', 'Noncomplex', 'Control']
res_cols = ['No. at Risk', 'COPD Diagnoses, n (%)', 'HR', '95% CI', 'P-Value']

for regtype, (data, form, strata) in formtypes.items():
    res_df = pd.DataFrame(index=groups, columns=res_cols)
    
    all_chd_form = form + ' + chd'
    subgroup_form = form + ' + complex_chd + noncomplex_chd + isolated_aov'
    
    cph1 = CoxPHFitter()
    cph1.fit(
        data,
        duration_col='yrs_duration',
        event_col='event',
        formula=all_chd_form,
        strata=strata)

    cph2 = CoxPHFitter()
    cph2.fit(
        data,
        duration_col='yrs_duration',
        event_col='event',
        formula=subgroup_form,
        strata=strata)
    
    reg_res = pd.concat((cph1.summary, cph2.summary))
    
    group_masks = {
        'All CHD': ('chd', data['chd'] == 1),
        'Noncomplex': ('noncomplex_chd', data['noncomplex_chd'] == 1),
        'Isolated AoV': ('isolated_aov', data['isolated_aov'] == 1),
        'Control': (None, data['chd'] == 0)
    }
    
    for group, (label, mask) in group_masks.items():
        events = data[mask & (data['event'] == 1)].index.size
        n = data[mask].index.size
        res_df.loc[group, 'No. at Risk'] = f'{n:,}'
        event_pc = 100 * events/n
        res_df.loc[group, 'COPD Diagnoses, n (%)'] = f'{events:,} ({round(event_pc, 1)})'
        if label:
            hr = round(reg_res.loc[label, 'exp(coef)'],2)
            hr_llim = round(reg_res.loc[label, 'exp(coef) lower 95%'],2)
            hr_ulim = round(reg_res.loc[label, 'exp(coef) upper 95%'],2)
            ci = f'({hr_llim}, {hr_ulim})'
            p = np.format_float_scientific(reg_res.loc[label, 'p'],2)
            
            res_df.loc[group, 'HR'] = hr
            res_df.loc[group, '95% CI'] = ci
            res_df.loc[group, 'P-Value'] = p
    
    results[regtype] = res_df

In [10]:
fa_res = results['Fully Adjusted'].T
fa_res

,All CHD,Isolated AoV,Noncomplex,Control
No. at Risk,"3,385","1,960","1,294","479,765"
"COPD Diagnoses, n (%)",294 (8.7),176 (9.0),106 (8.2),"14,854 (3.1)"
HR,2.18,2.1,2.35,NaN
95% CI,"(1.95, 2.45)","(1.81, 2.44)","(1.94, 2.84)",NaN
P-Value,6.3e-40,1.69e-22,2.51e-18,NaN
